In [1]:
import glob
import os
import sys
import struct
import pandas as pd
from nltk.tokenize import sent_tokenize
from tensorflow.core.example import example_pb2

sys.path.append('../src')
import data_io, params, SIF_embedding

In [2]:
inputfile = "/home/francisco/GitHub/DQN-Event-Summarization/data/cnn_tokenized/cnn_data_corpus.csv"
inputdict = "/home/francisco/GitHub/DQN-Event-Summarization/data/cnn_tokenized/cnn_total_corpus_smry.csv"

qdf = pd.read_csv(inputfile)
qdict = pd.read_csv(inputdict)
corpus_dict = dict(zip(qdict['id'], qdict['token']))

/home/francisco/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df1 = pd.read_csv('/home/francisco/GitHub/DQN-Event-Summarization/data/1-output/cnn_trainingstreams.csv')

In [4]:
df1.head()

,query_id,sentence_idx,label,query,sentence,true_summary
0,0,0,1,Mistaken for your child's grandmother,"-- i 'm 45 , and my son is 7","-- i 'm 45 , and my son is 7 once in a while ,..."
1,0,1,1,Mistaken for your child's grandmother,"once in a while , i still get carded when i tr...","-- i 'm 45 , and my son is 7 once in a while ,..."
2,0,2,1,Mistaken for your child's grandmother,i was 38 when Dominican Republic Emergency Ope...,"-- i 'm 45 , and my son is 7 once in a while ,..."
3,0,3,1,Mistaken for your child's grandmother,both incidents took place after i moved from A...,"-- i 'm 45 , and my son is 7 once in a while ,..."
4,0,4,2,Mistaken for your child's grandmother,i thought about the incidents when i read a re...,"-- i 'm 45 , and my son is 7 once in a while ,..."


In [9]:
'i thought about the incidents when i read' in df1[df1['query_id'] == 0]['true_summary'][0]

False

In [74]:
sdf = df1[df1['query_id'] == 0].reset_index(drop=True)

In [15]:
mainpath = '/home/francisco/GitHub/DQN-Event-Summarization/'

In [16]:
wordf = os.path.join(mainpath, 'data/glove.840B.300d.txt')
weightf = os.path.join(mainpath, 'auxiliary_data/enwiki_vocab_min200.txt')

myparams = params.params()
wp = 1e-3
rp = 0

Weights, words, word2weight, weight4ind = load_embed(wp, myparams, rp)

In [76]:
pd.options.display.max_rows = 125

In [77]:
sdf['sentence']

0                           -- i 'm 45 , and my son is 7
1      once in a while , i still get carded when i tr...
2      i was 38 when Dominican Republic Emergency Ope...
3      both incidents took place after i moved from A...
4      i thought about the incidents when i read a re...
5      the essay inspired me to post my case of mista...
6      had i let too many gray hairs escape my patrol...
7      but i have since learned about a convergence o...
8      first , experts say teenagers are having fewer...
9      " we believe evidence - based teen pregnancy p...
10             " we also believe in giving the kids hope
11     even with all the education in the world , if ...
12     and recent data shows that Sue Vannasing was o...
13                                it 's a national trend
14     according to preliminary data from the Cesar G...
15     black births to teens ages 15 to 17 have plumm...
16     still , black and London teenagers still have ...
17     the february report show

In [55]:

for i in range(sdf.shape[0]):
    tempsentence = sdf['sentence'][i]
    ab = sent_tokenize(tempsentence)
    clean_sentence = '. '.join([' '.join(s for s in x.split() if s.isalnum()) for x in ''.join(ab).replace("<s>","").split("</s>")]).strip()
    qembedding = return_sif([clean_sentence], words, weight4ind, myparams, Weights)
    if i == 0:
        edf = pd.DataFrame(qembedding)
    else:
        edf = pd.concat([edf, pd.DataFrame(qembedding)], axis=0)

KeyError: 61

In [52]:
edf.columns = ['embeding_%i' % i for i in range(edf.shape[1\])]

In [53]:
test = pd.concat([sdf, edf], axis=1)

In [54]:
test

,query_id,sentence_idx,label,query,sentence,true_summary,embeding_0,embeding_1,embeding_2,embeding_3,...,embeding_290,embeding_291,embeding_292,embeding_293,embeding_294,embeding_295,embeding_296,embeding_297,embeding_298,embeding_299
0,0,0,1,Mistaken for your child's grandmother,"-- i 'm 45 , and my son is 7","-- i 'm 45 , and my son is 7 once in a while ,...",-0.022753,0.17497,0.002637,-0.009615,...,0.015692,0.042637,-0.0263,0.116401,0.150211,-0.017192,0.028009,-0.019771,0.035994,-0.058288
1,0,1,1,Mistaken for your child's grandmother,"once in a while , i still get carded when i tr...","-- i 'm 45 , and my son is 7 once in a while ,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2,1,Mistaken for your child's grandmother,i was 38 when Dominican Republic Emergency Ope...,"-- i 'm 45 , and my son is 7 once in a while ,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,3,1,Mistaken for your child's grandmother,both incidents took place after i moved from A...,"-- i 'm 45 , and my son is 7 once in a while ,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,4,2,Mistaken for your child's grandmother,i thought about the incidents when i read a re...,"-- i 'm 45 , and my son is 7 once in a while ,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,5,2,Mistaken for your child's grandmother,the essay inspired me to post my case of mista...,"-- i 'm 45 , and my son is 7 once in a while ,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,6,2,Mistaken for your child's grandmother,had i let too many gray hairs escape my patrol...,"-- i 'm 45 , and my son is 7 once in a while ,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,7,1,Mistaken for your child's grandmother,but i have since learned about a convergence o...,"-- i 'm 45 , and my son is 7 once in a while ,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,8,2,Mistaken for your child's grandmother,"first , experts say teenagers are having fewer...","-- i 'm 45 , and my son is 7 once in a while ,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,9,1,Mistaken for your child's grandmother,""" we believe evidence - based teen pregnancy p...","-- i 'm 45 , and my son is 7 once in a while ,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


limit_output extension: Maximum message size of 10000 exceeded with 34624 characters

In [12]:
def return_bytes(reader_obj):
    len_bytes = reader_obj.read(8)
    str_len = struct.unpack('q', len_bytes)[0]
    e_s = struct.unpack("%ds" % str_len, reader_obj.read(str_len))
    es = e_s[0]
    c = example_pb2.Example.FromString(es)
    print(c.features.feature.keys())
    article  = str(c.features.feature['article'].bytes_list.value[0])
    abstract = str(c.features.feature['abstract'].bytes_list.value[0])
    ab = sent_tokenize(abstract)
    clean_article = sent_tokenize(article)
    clean_abstract = '. '.join([' '.join(s for s in x.split() if s.isalnum()) for x in ''.join(ab).replace("<s>","").split("</s>")]).strip()
    return clean_abstract, clean_article


def load_embed(weightpara=1e-3, param=None, rmpc=0):
    '''
    wordfile:   : location of embedding data (e.g., glove embedings)
    weightfile: : location of TF data for words
    weightpara: : the parameter in the SIF weighting scheme, usually in range [3e-5, 3e-3]
    rmpc:       : number of principal components to remove in SIF weighting scheme
    '''
    # input
    wordfile = '/home/francisco/GitHub/SIF/data/glove.840B.300d.txt'   # word vector file, can be downloaded from GloVe website
    weightfile = '/home/francisco/GitHub/SIF/auxiliary_data/enwiki_vocab_min200.txt' # each line is a word and its frequency

    # load word vectors
    (words, Weights) = data_io.getWordmap(wordfile)

    # load word weights
    word2weight = data_io.getWordWeight(weightfile, weightpara) # word2weight['str'] is the weight for the word 'str'
    weight4ind = data_io.getWeight(words, word2weight) # weight4ind[i] is the weight for the i-th word

    # set parameters
    param.rmpc = rmpc

    return Weights, words, word2weight, weight4ind

def return_sif(sentences, words, weight4ind, param, Weights):
    # x is the array of word indices, m is the binary mask indicating whether there is a word in that location
    x, m = data_io.sentences2idx(sentences, words)
    w = data_io.seq2weight(x, m, weight4ind) # get word weights
    # get SIF embedding
    embeddings = SIF_embedding.SIF_embedding(Weights, x, w, param) # embedding[i,:] is the embedding for sentence i
    return embeddings


def embed_sentences(inputpath, wordfile, weightfile, weightpara, param, rmpc, file_list):
    Weights, words, word2weight, weight4ind = load_embed(wordfile, weightfile, weightpara, param, rmpc)

    print('embeddings loaded...')
    for file_i in file_list:
        input_file = open(os.path.join(inputpath, file_i), 'rb')
        c = 0
        while input_file:
            try:
                clean_abstract, clean_article = return_bytes(input_file)
            except:
                input_file = None

            print('article cleaned...')
            break
            embeddings = return_sif(clean_article, words, weight4ind, param, Weights)

            sdf = pd.DataFrame(clean_article, columns=['sentence'])
            sdf['clean_sentence'] = [' '.join([s for s in x if s.isalnum()]) for x in sdf['sentence'].str.split(" ")]
            sdf['summary'] = clean_abstract
            sdf.ix[1:, 'summary'] = ''

            embcols = ['emb_%i'%i for i in range(embeddings.shape[1])]
            emb = pd.DataFrame(embeddings, columns = embcols)

            sdf = pd.concat([sdf, emb], axis=1)
            sdf = sdf[['summary', 'sentence', 'clean_sentence'] + sdf.columns[3:].tolist()]
            newfile = file_i.replace(".bin", "").split("/")[-1]
            #sdf.to_csv("/home/francisco/GitHub/DQN-Event-Summarization/data/sif/%s_%i.csv" % (
            #         newfile, c
            #         )
            #    )
            sdf.to_csv("/home/francisco/GitHub/DQN-Event-Summarization/data/testsif/%s_%i.csv" % (
                     newfile, c
                     )
                )
            if (c % 100) == 0:
                 print("Data exported to %s_%i.csv" % (newfile, c))
            c+= 1

In [ ]:
myparams = params.params()
mainpath = 'home/francisco/GitHub/SIF/'
datapath = '/home/francisco/GitHub/cnn-dailymail/finished_files/chunked/'
wordf = os.path.join(mainpath, 'data/glove.840B.300d.txt')
weightf = os.path.join(mainpath, 'auxiliary_data/enwiki_vocab_min200.txt')
wp = 1e-3
rp = 0
# Example case
fl = ['/home/francisco/GitHub/cnn-dailymail/finished_files/chunked/train_000.bin']
#fl = os.listdir(datapath)
embed_sentences(datapath, wordf, weightf, wp, myparams, rp, fl)